# Librerias

In [1]:
import pandas as pd
import numpy as np

import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer

from sklearn.impute import SimpleImputer

from scikitplot.metrics import plot_roc_curve

from scipy.stats import ks_2samp

import seaborn as sns

In [2]:
pd.set_option('display.max_columns',55) 

# Extraccion de datos

In [3]:
path = '/home/david/Ciencia_Datos/'

In [4]:
import os 

In [5]:
lst_files = os.listdir( path )
lst_files = [ os.path.join(path,f) for f in lst_files ]
lst_files = [f for f in lst_files if f[-3:] == 'csv']

In [5]:
df = pd.read_csv( lst_files[0] )

In [6]:
df.shape[0]

101766

# Limpieza de datos

In [7]:
df.dtypes

id                           int64
encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
weight                      object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
payer_code                  object
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide      

In [8]:
df.columns

Index(['id', 'encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide.metformin', 'glipizide.metformin',
       'glimepiride.pioglitazone', 'metformin.rosiglitazone',
       'metformin.pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

# Variables

In [9]:
df.head(3)

,id,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,1,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,2,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,3,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO


In [10]:
um   = ['id']
# discreta
vard = ['race', 'gender', 'age','weight','admission_type_id','discharge_disposition_id',
        'admission_source_id' ,'payer_code', 'medical_specialty','max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide.metformin', 'glipizide.metformin',
       'glimepiride.pioglitazone', 'metformin.rosiglitazone',
       'metformin.pioglitazone', 'change', 'readmitted']
# continua
varc = ['time_in_hospital','num_lab_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'diag_1',
       'diag_2', 'diag_3','number_inpatient','number_diagnoses']
tgt = ['diabetesMed']

In [11]:
df['diabetesMed'].unique()

array(['No', 'Yes'], dtype=object)

In [12]:
df['diabetesMed'] = df['diabetesMed'].map(lambda x: 0 if x=='No' else 1)

In [13]:
df[df['diabetesMed']==1]['insulin'].unique()

array(['Up', 'No', 'Steady', 'Down'], dtype=object)

In [14]:
df['id'] = df['id'].astype(str)

# Analisis Exploratorio

## Discreto

### Frecuencias

In [15]:
def freq(df,var):
    
    if type(var) != list:
        var = [var]
    
    for v in var:
        aux = df[v].value_counts().to_frame().rename(columns={v:'FA'})
        aux['FR'] = aux['FA'] / aux['FA'].sum()
        aux[['FAA','FRA']] = aux.apply(np.cumsum )
        print(f"Frecuencias para la variable {v} \n")
        display(aux)
        print("\n")

In [16]:
freq(df,vard) 

Frecuencias para la variable race 



,FA,FR,FAA,FRA
Caucasian,76099,0.747784,76099,0.747784
AfricanAmerican,19210,0.188766,95309,0.936551
?,2273,0.022336,97582,0.958886
Hispanic,2037,0.020017,99619,0.978903
Other,1506,0.014799,101125,0.993701
Asian,641,0.006299,101766,1.000000




Frecuencias para la variable gender 



,FA,FR,FAA,FRA
Female,54708,0.537586,54708,0.537586
Male,47055,0.462384,101763,0.999971
Unknown/Invalid,3,0.000029,101766,1.000000




Frecuencias para la variable age 



...

,FA,FR,FAA,FRA
No,101765,0.99999,101765,0.99999
Steady,1,0.00001,101766,1.00000




Frecuencias para la variable change 



,FA,FR,FAA,FRA
No,54755,0.538048,54755,0.538048
Ch,47011,0.461952,101766,1.000000




Frecuencias para la variable readmitted 



,FA,FR,FAA,FRA
NO,54864,0.539119,54864,0.539119
>30,35545,0.349282,90409,0.888401
<30,11357,0.111599,101766,1.000000


### Normalizar

In [17]:
def normalizar(df,v,umbral):
    aux = df[v].value_counts(True).to_frame()
    aux[f'norm_{v}'] = np.where( aux[v] < umbral , 'CATEGORIAS PEQUEÑAS' , aux.index )

    moda = aux.head(1)[f'norm_{v}'].values[0]

    if aux.loc[aux[f'norm_{v}'] == 'CATEGORIAS PEQUEÑAS' ][v].sum() < umbral:
        aux[f'norm_{v}'].replace( {'CATEGORIA PEQUEÑAS':moda} , inplace=True)

    aux.drop(v, axis=1, inplace=True)
    aux.reset_index(inplace=True)

    return df.merge(aux,left_on=[v], right_on='index', how='inner').drop('index',axis=1)

In [18]:
for v in vard:
    df = normalizar(df,v,0.05)

In [19]:
varn = df.filter(like='norm_').columns.tolist()
varn.sort()

In [20]:
varn 

['norm_A1Cresult',
 'norm_acarbose',
 'norm_acetohexamide',
 'norm_admission_source_id',
 'norm_admission_type_id',
 'norm_age',
 'norm_change',
 'norm_chlorpropamide',
 'norm_citoglipton',
 'norm_discharge_disposition_id',
 'norm_examide',
 'norm_gender',
 'norm_glimepiride',
 'norm_glimepiride.pioglitazone',
 'norm_glipizide',
 'norm_glipizide.metformin',
 'norm_glyburide',
 'norm_glyburide.metformin',
 'norm_insulin',
 'norm_max_glu_serum',
 'norm_medical_specialty',
 'norm_metformin',
 'norm_metformin.pioglitazone',
 'norm_metformin.rosiglitazone',
 'norm_miglitol',
 'norm_nateglinide',
 'norm_payer_code',
 'norm_pioglitazone',
 'norm_race',
 'norm_readmitted',
 'norm_repaglinide',
 'norm_rosiglitazone',
 'norm_tolazamide',
 'norm_tolbutamide',
 'norm_troglitazone',
 'norm_weight']

In [21]:
freq(df,varn)

Frecuencias para la variable norm_A1Cresult 



,FA,FR,FAA,FRA
None,84748,0.832773,84748,0.832773
CATEGORIAS PEQUEÑAS,8802,0.086493,93550,0.919266
>8,8216,0.080734,101766,1.000000




Frecuencias para la variable norm_acarbose 



,FA,FR,FAA,FRA
No,101458,0.996973,101458,0.996973
CATEGORIAS PEQUEÑAS,308,0.003027,101766,1.000000




Frecuencias para la variable norm_acetohexamide 



...

,FA,FR,FAA,FRA
No,101743,0.999774,101743,0.999774
CATEGORIAS PEQUEÑAS,23,0.000226,101766,1.000000




Frecuencias para la variable norm_troglitazone 



,FA,FR,FAA,FRA
No,101763,0.999971,101763,0.999971
CATEGORIAS PEQUEÑAS,3,0.000029,101766,1.000000




Frecuencias para la variable norm_weight 



,FA,FR,FAA,FRA
?,98569,0.968585,98569,0.968585
CATEGORIAS PEQUEÑAS,3197,0.031415,101766,1.000000


### Unarias

In [22]:
unarias = [ v for v,conteo in zip( varn , [ df[v].unique().shape[0] for v in varn ] ) if conteo == 1 ]

In [23]:
unarias 

['norm_citoglipton', 'norm_examide']

In [24]:
varn = [v for v in varn  if v not in unarias]

In [25]:
varn 

['norm_A1Cresult',
 'norm_acarbose',
 'norm_acetohexamide',
 'norm_admission_source_id',
 'norm_admission_type_id',
 'norm_age',
 'norm_change',
 'norm_chlorpropamide',
 'norm_discharge_disposition_id',
 'norm_gender',
 'norm_glimepiride',
 'norm_glimepiride.pioglitazone',
 'norm_glipizide',
 'norm_glipizide.metformin',
 'norm_glyburide',
 'norm_glyburide.metformin',
 'norm_insulin',
 'norm_max_glu_serum',
 'norm_medical_specialty',
 'norm_metformin',
 'norm_metformin.pioglitazone',
 'norm_metformin.rosiglitazone',
 'norm_miglitol',
 'norm_nateglinide',
 'norm_payer_code',
 'norm_pioglitazone',
 'norm_race',
 'norm_readmitted',
 'norm_repaglinide',
 'norm_rosiglitazone',
 'norm_tolazamide',
 'norm_tolbutamide',
 'norm_troglitazone',
 'norm_weight']

## Continuas

### Ausentes

In [26]:
for c in ['diag_1','diag_2', 'diag_3']:
    print( c , df[c].map(type).unique().tolist())

diag_1 [<class 'str'>]
diag_2 [<class 'str'>]
diag_3 [<class 'str'>]


In [27]:
df['diag_1'] = df['diag_1'].astype(str)
df['diag_2'] = df['diag_2'].astype(str)
df['diag_3'] = df['diag_3'].astype(str)

In [28]:
for c in ['diag_1','diag_2', 'diag_3']:
    print( c , df[c].map(type).unique().tolist())

diag_1 [<class 'str'>]
diag_2 [<class 'str'>]
diag_3 [<class 'str'>]


In [29]:
for c in ['diag_1','diag_2', 'diag_3']:
    df[c] = df[c].map(lambda x: '?' if x[0]=='V' else x)
    df[c] = df[c].map(lambda x: '?' if x[0]=='E' else x)


In [30]:
for c in varc:
    df[c].loc[df[c] == '?'] = np.nan

/tmp/ipykernel_21913/1682592610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[c].loc[df[c] == '?'] = np.nan


In [31]:
for c in ['diag_1','diag_2', 'diag_3']:
    df[c] = df[c].astype(float)

In [32]:
miss = 1 - df[varc].describe().T[['count']] / len(df)
miss

,count
time_in_hospital,0.000000
num_lab_procedures,0.000000
num_medications,0.000000
number_outpatient,0.000000
number_emergency,0.000000
diag_1,0.016371
diag_2,0.028438
diag_3,0.063685
number_inpatient,0.000000
number_diagnoses,0.000000


In [33]:
x = df[varc]

In [34]:
im = SimpleImputer(strategy='median')

In [35]:
im.fit(x)

SimpleImputer(strategy='median')

In [36]:
x[varc] = im.transform(x[varc])

/tmp/ipykernel_21913/1148555675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[varc] = im.transform(x[varc])


In [37]:
x 

,time_in_hospital,num_lab_procedures,num_medications,number_outpatient,number_emergency,diag_1,diag_2,diag_3,number_inpatient,number_diagnoses
0,1.0,41.0,1.0,0.0,0.0,250.83,425.0,403.0,0.0,1.0
1,2.0,13.0,1.0,0.0,0.0,250.03,278.0,403.0,0.0,2.0
2,2.0,2.0,1.0,0.0,0.0,250.00,279.0,701.0,0.0,4.0
3,2.0,3.0,3.0,0.0,0.0,250.02,276.0,79.0,0.0,3.0
4,1.0,50.0,8.0,0.0,0.0,250.13,276.0,403.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...
101761,5.0,36.0,20.0,3.0,0.0,250.60,197.0,157.0,1.0,8.0
101762,4.0,29.0,11.0,0.0,0.0,682.00,250.7,41.0,1.0,9.0
101763,4.0,30.0,24.0,2.0,0.0,250.80,707.0,403.0,3.0,2.0
101764,4.0,1.0,4.0,0.0,0.0,812.00,250.0,781.0,0.0,4.0


In [38]:
ks = pd.DataFrame( map( lambda v: ( v , ks_2samp( df[v].dropna() , x[v] ).statistic ) , varc  ) , columns = ['var','ks']   )
ks 

,var,ks
0,time_in_hospital,0.000000
1,num_lab_procedures,0.000000
2,num_medications,0.000000
3,number_outpatient,0.000000
4,number_emergency,0.000000
5,diag_1,0.008170
6,diag_2,0.014213
7,diag_3,0.031084
8,number_inpatient,0.000000
9,number_diagnoses,0.000000


In [39]:
# Sin alterados

In [40]:
df[varc] = im.transform(df[varc])

### Discretizar

In [41]:
df = df.rename(columns={'diag_1':'diag1',
                  'diag_2':'diag2',
                  'diag_3':'diag3'})

In [42]:
def discretizar(df, v, k):

    kb = KBinsDiscretizer( n_bins = k , encode='ordinal', strategy='quantile')
    kb.fit(df[[v]])
    df[f'd_{v}_{k}'] = pd.cut( df[v] , bins=kb.bin_edges_[0] , include_lowest=True ).astype(str)

    return df

In [43]:
varc 

['time_in_hospital',
 'num_lab_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'diag_1',
 'diag_2',
 'diag_3',
 'number_inpatient',
 'number_diagnoses']

In [44]:
varc  = ['time_in_hospital',
 'num_lab_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'diag1',
 'diag2',
 'diag3',
 'number_inpatient',
 'number_diagnoses']

In [45]:
for v in varc:
    for k in range(2,6):
        df = discretizar( df , v , k )

/home/david/AMV/dave/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:291: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/david/AMV/dave/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:291: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/david/AMV/dave/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:291: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/david/AMV/dave/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:291: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/david/AMV/dave/lib/python3.10/site-pac

In [46]:
vari = df.filter(like='d_').columns.to_list()

In [47]:
vari 

['d_time_in_hospital_2',
 'd_time_in_hospital_3',
 'd_time_in_hospital_4',
 'd_time_in_hospital_5',
 'd_num_lab_procedures_2',
 'd_num_lab_procedures_3',
 'd_num_lab_procedures_4',
 'd_num_lab_procedures_5',
 'd_num_medications_2',
 'd_num_medications_3',
 'd_num_medications_4',
 'd_num_medications_5',
 'd_number_outpatient_2',
 'd_number_outpatient_3',
 'd_number_outpatient_4',
 'd_number_outpatient_5',
 'd_number_emergency_2',
 'd_number_emergency_3',
 'd_number_emergency_4',
 'd_number_emergency_5',
 'd_diag1_2',
 'd_diag1_3',
 'd_diag1_4',
 'd_diag1_5',
 'd_diag2_2',
 'd_diag2_3',
 'd_diag2_4',
 'd_diag2_5',
 'd_diag3_2',
 'd_diag3_3',
 'd_diag3_4',
 'd_diag3_5',
 'd_number_inpatient_2',
 'd_number_inpatient_3',
 'd_number_inpatient_4',
 'd_number_inpatient_5',
 'd_number_diagnoses_2',
 'd_number_diagnoses_3',
 'd_number_diagnoses_4',
 'd_number_diagnoses_5']

## IV Funcion

In [48]:
def calculo_iv( df, v, tgt ):
    aux = df.pivot_table( index = v, 
                         columns= tgt, 
                         values = um[0], 
                         aggfunc= 'count', 
                         fill_value=0 )

    aux[list( range(2) ) ] = aux/aux.apply(np.sum)

    aux['w'] = np.log( aux[0] / aux[1] )
    aux['iv'] = (aux[0] - aux[1]) * aux['w']
    
    return v, aux['iv'].sum() 

# Particion de datos

## Mejores variables

In [49]:
X = df[ vari + ['id'] ].copy()
y = df[ tgt  + ['id'] ].copy()

Xt, Xv, yt, yv = train_test_split(X, y, train_size=.7, random_state=2000 )

In [50]:
Xt = Xt.merge( yt , on=um, how='inner').reset_index(drop=True)

In [51]:
# iv de discretizadas -> vari
iv = pd.DataFrame( map( lambda v: calculo_iv( df, v, tgt  ) , vari ) , 
                  columns=['variable','iv'] )

iv['raiz'] = iv['variable'].map( lambda x: '_'.join(x.split('_')[1:2] ) )
iv = iv.sort_values(by=['raiz','iv'],ascending=[1,0]).reset_index(drop=True)

# Eliminación de ivs infs
iv = iv.loc[iv['iv'] != np.inf].reset_index(drop=True)
iv['id'] = iv.groupby('raiz').cumcount()+1
iv = iv.loc[iv.id == 1]

iv.drop(columns=['raiz','id'],inplace=True)

In [52]:
iv

,variable,iv
0,d_diag1_5,0.014494
4,d_diag2_4,0.003711
8,d_diag3_4,0.001103
12,d_num_medications_5,0.218622
20,d_number_diagnoses_5,0.004292
36,d_time_in_hospital_5,0.027428


In [53]:
besti = iv['variable'].tolist()
besti

['d_diag1_5',
 'd_diag2_4',
 'd_diag3_4',
 'd_num_medications_5',
 'd_number_diagnoses_5',
 'd_time_in_hospital_5']

In [54]:
X[besti]

,d_diag1_5,d_diag2_4,d_diag3_4,d_num_medications_5,d_number_diagnoses_5,d_time_in_hospital_5
0,"(2.999, 296.0]","(278.0, 425.0]","(276.0, 403.0]","(0.999, 9.0]","(0.999, 5.0]","(0.999, 2.0]"
1,"(2.999, 296.0]","(4.999, 278.0]","(276.0, 403.0]","(0.999, 9.0]","(0.999, 5.0]","(0.999, 2.0]"
2,"(2.999, 296.0]","(278.0, 425.0]","(493.75, 999.0]","(0.999, 9.0]","(0.999, 5.0]","(0.999, 2.0]"
3,"(2.999, 296.0]","(4.999, 278.0]","(2.999, 276.0]","(0.999, 9.0]","(0.999, 5.0]","(0.999, 2.0]"
4,"(2.999, 296.0]","(4.999, 278.0]","(276.0, 403.0]","(0.999, 9.0]","(0.999, 5.0]","(0.999, 2.0]"
...,...,...,...,...,...,...
101761,"(2.999, 296.0]","(4.999, 278.0]","(2.999, 276.0]","(17.0, 22.0]","(7.0, 9.0]","(4.0, 7.0]"
101762,"(491.0, 682.0]","(4.999, 278.0]","(2.999, 276.0]","(9.0, 13.0]","(7.0, 9.0]","(3.0, 4.0]"
101763,"(2.999, 296.0]","(518.0, 999.0]","(276.0, 403.0]","(22.0, 81.0]","(0.999, 5.0]","(3.0, 4.0]"
101764,"(682.0, 999.0]","(4.999, 278.0]","(493.75, 999.0]","(0.999, 9.0]","(0.999, 5.0]","(3.0, 4.0]"


## Variables para Scoring

In [55]:
X = df[ vari + ['id'] ].copy()
y = df[ tgt  + ['id'] ].copy()

Xt, Xv, yt, yv = train_test_split(X, y, train_size=.7, random_state=2000 )

print(Xt.shape, Xv.shape, yt.shape[0], yv.shape[0])

Xt = Xt.merge( yt , on=um, how='inner').reset_index(drop=True)

(71236, 41) (30530, 41) 71236 30530


In [56]:
# iv de normalizadas -> varn
ivd = pd.DataFrame( map( lambda v: calculo_iv( df, v, tgt  ) , varn ) , 
                   columns=['variable','iv'] )

/home/david/AMV/dave/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/david/AMV/dave/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/david/AMV/dave/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/david/AMV/dave/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/david/AMV/dave/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/david/AMV/dave/lib/python3.10/site-packages/pandas/core/arraylike.py:405: Runti

In [57]:
ivd 

,variable,iv
0,norm_A1Cresult,0.065663
1,norm_acarbose,inf
2,norm_acetohexamide,inf
3,norm_admission_source_id,0.000103
4,norm_admission_type_id,0.010324
5,norm_age,0.004414
6,norm_change,inf
7,norm_chlorpropamide,inf
8,norm_discharge_disposition_id,0.017673
9,norm_gender,0.001444


In [58]:
ivd = ivd[ivd['iv'] !=np.inf ]

In [59]:
iv = pd.concat([ iv, ivd ], ignore_index=True)

In [60]:
iv 

,variable,iv
0,d_diag1_5,0.014494
1,d_diag2_4,0.003711
2,d_diag3_4,0.001103
3,d_num_medications_5,0.218622
4,d_number_diagnoses_5,0.004292
5,d_time_in_hospital_5,0.027428
6,norm_A1Cresult,0.065663
7,norm_admission_source_id,0.000103
8,norm_admission_type_id,0.010324
9,norm_age,0.004414


In [61]:
best = iv['variable'].tolist()
len(best)

18

In [62]:
df.columns 

Index(['id', 'encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       ...
       'd_diag3_4', 'd_diag3_5', 'd_number_inpatient_2',
       'd_number_inpatient_3', 'd_number_inpatient_4', 'd_number_inpatient_5',
       'd_number_diagnoses_2', 'd_number_diagnoses_3', 'd_number_diagnoses_4',
       'd_number_diagnoses_5'],
      dtype='object', length=127)

# Transformacion WoE

In [63]:
X = df[ best + ['id'] ].copy()
y = df[ tgt  + ['id'] ].copy()

Xt, Xv, yt, yv = train_test_split(X, y, train_size=.7, random_state=2000 )



Xt = Xt.merge( yt , on=um, how='inner').reset_index(drop=True)

## Codificacion WoE

In [64]:
def codificacion_woe(df, v, tgt):
    aux = df.pivot_table( index=v,
                          columns=tgt,
                          values=um[0],
                          aggfunc='count',
                          fill_value=0)

    aux[list( range(2) )] = aux / aux.apply(np.sum)

    aux['w'] = np.log( aux[0] / aux[1] )

    aux.drop(range(2) , axis=1 , inplace=True )

    aux = aux.to_dict()['w']
    
    return v, aux

In [65]:
mapa_woe = list( map(  lambda v: codificacion_woe( Xt , v , tgt  ) , best  ) )

In [66]:
for v, mapa in mapa_woe:
    Xt[f'w_{v}'] = Xt[v].replace(mapa)
    Xv[f'w_{v}'] = Xv[v].replace(mapa)

In [67]:
print(Xt.shape, Xv.shape, yt.shape[0], yv.shape[0])

(71236, 38) (30530, 37) 71236 30530


In [68]:
varw = Xt.filter(like='w_').columns.tolist() 

# TAD

In [69]:
tad_t = Xt[ um + varw  ].merge( yt , on = um , how='inner'  )

In [70]:
tad_t

,id,w_d_diag1_5,w_d_diag2_4,w_d_diag3_4,w_d_num_medications_5,w_d_number_diagnoses_5,w_d_time_in_hospital_5,w_norm_A1Cresult,w_norm_admission_source_id,w_norm_admission_type_id,w_norm_age,w_norm_discharge_disposition_id,w_norm_gender,w_norm_max_glu_serum,w_norm_medical_specialty,w_norm_payer_code,w_norm_race,w_norm_readmitted,w_norm_weight,diabetesMed
0,78520,0.038626,-0.086659,-0.038218,-0.170157,-0.049121,-0.113310,0.071156,0.003838,0.035555,-0.028127,-0.152203,-0.042718,-0.00961,0.057270,-0.155742,0.008101,0.126184,-0.015095,1
1,7252,-0.140094,0.080783,0.037102,-0.170157,0.026087,-0.113310,0.007783,0.003838,0.035555,0.093634,-0.016658,0.035862,-0.00961,0.057270,0.204086,0.005909,0.126184,-0.015095,0
2,98578,0.198189,-0.086659,0.027807,-0.348349,-0.049121,-0.032322,-1.024311,-0.013811,-0.091907,-0.072729,-0.016658,0.035862,-0.00961,0.023887,-0.261081,0.008101,-0.149852,-0.015095,1
3,75446,-0.015715,-0.049035,0.037102,-0.691886,-0.049121,-0.203704,0.071156,0.003838,0.035555,0.113007,0.271310,0.035862,-0.00961,0.057270,-0.155742,0.008101,0.126184,-0.015095,0
4,28631,0.038626,0.080783,0.037102,-0.170157,0.026087,-0.113310,0.071156,0.038589,0.035555,-0.004411,-0.016658,-0.042718,-0.00961,-0.067686,0.251925,-0.116190,-0.185172,-0.015095,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,17405,-0.015715,-0.049035,-0.027331,-0.170157,0.112506,-0.088500,0.071156,0.003838,0.035555,0.113007,0.271310,-0.042718,-0.00961,0.057270,0.204086,0.008101,0.126184,-0.015095,1
71232,19763,-0.122419,-0.086659,0.027807,0.700748,0.026087,-0.032322,0.071156,0.003838,0.035555,-0.072729,-0.016658,-0.042718,-0.00961,0.057270,0.204086,0.005909,-0.149852,-0.015095,1
71233,39926,0.038626,0.044496,0.037102,-0.170157,-0.049121,-0.032322,0.071156,-0.013811,-0.030932,-0.028127,-0.016658,0.035862,-0.00961,0.057270,-0.155742,0.008101,-0.149852,0.412987,1
71234,24332,0.038626,-0.049035,0.037102,-0.170157,0.026087,-0.088500,0.071156,-0.013811,-0.091907,-0.072729,-0.016658,-0.042718,-0.00961,0.057270,-0.155742,0.008101,0.126184,-0.015095,1


In [78]:
pd.to_pickle( tad_t , 'Entrega_Modelo.pkl' , protocol=4  )  